In [8]:
import pandas as pd


In [9]:
# Input and output file paths
input_file = "uscities_with_aige.csv"  # Input CSV file
output_file = "us_states_aige.csv"  # Output CSV file
# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Ensure the required columns are present
columns_to_focus = ["state_id", "state_name", "population", "AIGE"]
for column in columns_to_focus:
    if column not in df.columns:
        raise ValueError(f"Missing required column: {column}")


In [10]:
# Convert 'population' and 'AIGE' columns to numeric types for calculations
df["population"] = pd.to_numeric(df["population"], errors="coerce")
df["AIGE"] = pd.to_numeric(df["AIGE"], errors="coerce")

In [11]:
# Group by 'state_id' and 'state_name' to calculate the weighted average AIGE for each state
state_summary = df.groupby(["state_id", "state_name"]).apply(
    lambda group: pd.Series({
        "AIGE": (group["AIGE"] * group["population"]).sum() / group["population"].sum(),
    })
).reset_index()

C:\Users\joanna\AppData\Local\Temp\ipykernel_6276\2657614793.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  state_summary = df.groupby(["state_id", "state_name"]).apply(


In [12]:
# Calculate the total weighted average AIGE for the entire country
total_population = df["population"].sum()  # Sum of the population across all cities
total_aige = (df["AIGE"] * df["population"]).sum() / total_population  # Weighted average AIGE

# Append the national average as the last row of the new DataFrame
state_summary = pd.concat(
    [state_summary, pd.DataFrame({
        "state_id": ["US"],  # National identifier
        "state_name": ["United States"],  # Name for the entire country
        "AIGE": [total_aige],  # National weighted average AIGE
    })],
    ignore_index=True  # Reset the index for the new DataFrame
)


In [13]:
# Save the resulting DataFrame to a new CSV file
state_summary.to_csv(output_file, index=False)

print(f"Calculation completed. The new file has been saved as {output_file}")

Calculation completed. The new file has been saved as us_states_aige.csv
